# SMR Performance Analysis

Interactive analysis of Safe Memory Reclamation (SMR) behavior using `SMRProfiler`.

This notebook helps you:
- Understand epoch advancement patterns
- Identify stalled threads blocking reclamation
- Analyze reclamation latency
- Monitor memory pressure
- Optimize SMR configuration

## 1. Setup

In [ ]:
import sys
import threading
import time
import random
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional, List

try:
    from concurrent_collections import SMRProfiler, SkipListMap, config
    LIBRARY_AVAILABLE = True
    print(f"concurrent_collections loaded")
    print(f"SMR scheme: {config.smr_scheme}")
except ImportError:
    LIBRARY_AVAILABLE = False
    print("Running in simulation mode")

## 2. Simulation Mode

In [ ]:
if not LIBRARY_AVAILABLE:
    @dataclass
    class ThreadSMRStats:
        thread_id: int
        thread_name: Optional[str]
        enter_count: int
        exit_count: int
        retire_count: int
        poll_count: int
        reclaim_count: int
        total_cs_time_ns: int
        avg_cs_time_ns: float
        max_cs_time_ns: int
        peak_limbo_count: int
        peak_limbo_bytes: int
        stall_count: int
        caused_stall_epochs: int
    
    @dataclass
    class StallEvent:
        timestamp: float
        stalled_thread_id: int
        stalled_at_epoch: int
        global_epoch: int
        epoch_lag: int
        duration_ns: Optional[int]
        resolution: str
    
    @dataclass
    class SMRProfilerReport:
        start_epoch: int
        end_epoch: int
        epoch_advances: int
        avg_epoch_duration_ns: float
        safe_epoch_lag_avg: float
        safe_epoch_lag_max: int
        total_retired: int
        total_reclaimed: int
        pending_count: int
        pending_bytes: int
        reclaim_latency_p50: float
        reclaim_latency_p95: float
        reclaim_latency_p99: float
        reclaim_latency_p999: float
        reclaim_latency_max: float
        poll_count: int
        nodes_per_poll_avg: float
        nodes_per_poll_max: int
        empty_poll_pct: float
        peak_pending_count: int
        peak_pending_bytes: int
        memory_bound_utilization: float
        thread_stats: List[ThreadSMRStats]
        stall_events: List[StallEvent]
        total_stall_time_ns: int
        stall_count: int
        epoch_timeline: Optional[list]
        limbo_snapshots: Optional[list]
        duration_seconds: float
        start_time: datetime
        end_time: datetime

    def generate_simulated_smr_report():
        """Generate realistic simulated SMR data."""
        thread_stats = [
            ThreadSMRStats(1, "MainThread", 50000, 50000, 25000, 500, 24500, 
                          500000000, 10000, 150000, 128, 8192, 0, 0),
            ThreadSMRStats(2, "Worker-1", 45000, 45000, 22000, 450, 21500,
                          450000000, 10000, 120000, 115, 7360, 1, 5),
            ThreadSMRStats(3, "Worker-2", 48000, 48000, 24000, 480, 23500,
                          480000000, 10000, 180000, 122, 7808, 0, 0),
            ThreadSMRStats(4, "Worker-3", 42000, 42000, 21000, 420, 20500,
                          420000000, 10000, 200000, 108, 6912, 2, 12),
            ThreadSMRStats(5, "Worker-4", 46000, 46000, 23000, 460, 22500,
                          460000000, 10000, 140000, 118, 7552, 0, 0),
            ThreadSMRStats(6, "Worker-5", 44000, 44000, 22000, 440, 21500,
                          440000000, 10000, 160000, 112, 7168, 1, 3),
            ThreadSMRStats(7, "Worker-6", 47000, 47000, 23500, 470, 23000,
                          470000000, 10000, 130000, 120, 7680, 0, 0),
            ThreadSMRStats(8, "Worker-7", 43000, 43000, 21500, 430, 21000,
                          430000000, 10000, 170000, 110, 7040, 1, 2),
        ]
        
        stall_events = [
            StallEvent(1.234, 2, 1000, 1015, 15, 50000000, "exited"),
            StallEvent(3.456, 4, 2500, 2520, 20, 80000000, "exited"),
            StallEvent(5.678, 4, 4000, 4018, 18, 60000000, "exited"),
            StallEvent(7.890, 6, 5500, 5512, 12, 40000000, "exited"),
            StallEvent(9.012, 8, 7000, 7010, 10, 30000000, "exited"),
        ]
        
        return SMRProfilerReport(
            start_epoch=1,
            end_epoch=12345,
            epoch_advances=12344,
            avg_epoch_duration_ns=810500,
            safe_epoch_lag_avg=1.2,
            safe_epoch_lag_max=20,
            total_retired=182000,
            total_reclaimed=181950,
            pending_count=50,
            pending_bytes=3200,
            reclaim_latency_p50=5000,
            reclaim_latency_p95=25000,
            reclaim_latency_p99=100000,
            reclaim_latency_p999=500000,
            reclaim_latency_max=2000000,
            poll_count=3600,
            nodes_per_poll_avg=50.5,
            nodes_per_poll_max=128,
            empty_poll_pct=12.5,
            peak_pending_count=512,
            peak_pending_bytes=32768,
            memory_bound_utilization=0.67,
            thread_stats=thread_stats,
            stall_events=stall_events,
            total_stall_time_ns=260000000,
            stall_count=5,
            epoch_timeline=None,
            limbo_snapshots=None,
            duration_seconds=10.0,
            start_time=datetime.now(),
            end_time=datetime.now(),
        )
    
    print("Simulation classes created")

## 3. Workload Configuration

In [ ]:
WORKLOAD_CONFIG = {
    'num_threads': 8,
    'operations_per_thread': 50_000,
    'key_range': 25_000,
    'delete_ratio': 0.3,  # Higher delete ratio to stress SMR
    'stall_probability': 0.001,  # 0.1% chance of brief stall
    'stall_duration_ms': 5,
}

print("Workload Configuration:")
for key, value in WORKLOAD_CONFIG.items():
    print(f"  {key}: {value}")

## 4. Run Profiled Workload

In [ ]:
if LIBRARY_AVAILABLE:
    def run_smr_workload():
        m = SkipListMap()
        
        def worker(thread_id):
            for i in range(WORKLOAD_CONFIG['operations_per_thread']):
                key = f"key_{random.randint(0, WORKLOAD_CONFIG['key_range'])}"
                
                # Occasional stall to trigger SMR pressure
                if random.random() < WORKLOAD_CONFIG['stall_probability']:
                    time.sleep(WORKLOAD_CONFIG['stall_duration_ms'] / 1000)
                
                if random.random() < WORKLOAD_CONFIG['delete_ratio']:
                    m.pop(key, None)  # Delete triggers SMR retire
                else:
                    m[key] = f"value_{i}"
        
        with SMRProfiler(
            track_per_thread=True,
            track_latency=True,
            stall_threshold_epochs=10,
        ) as prof:
            threads = [
                threading.Thread(target=worker, args=(i,), name=f"Worker-{i}")
                for i in range(WORKLOAD_CONFIG['num_threads'])
            ]
            
            for t in threads:
                t.start()
            for t in threads:
                t.join()
        
        return prof.report()
    
    print("Running SMR workload...")
    report = run_smr_workload()
    print(f"Completed in {report.duration_seconds:.2f} seconds")
else:
    print("Using simulated data...")
    report = generate_simulated_smr_report()

## 5. Epoch Analysis

In [ ]:
print("=" * 60)
print("SMR PROFILER RESULTS")
print("=" * 60)

print(f"\n### Epoch Statistics ###")
print(f"Start epoch: {report.start_epoch:,}")
print(f"End epoch: {report.end_epoch:,}")
print(f"Epoch advances: {report.epoch_advances:,}")
print(f"Avg epoch duration: {report.avg_epoch_duration_ns / 1000:.1f} µs")
print(f"Epoch advance rate: {report.epoch_advances / report.duration_seconds:.0f} epochs/sec")

print(f"\n### Safe Epoch Lag ###")
print(f"Average lag: {report.safe_epoch_lag_avg:.2f} epochs")
print(f"Maximum lag: {report.safe_epoch_lag_max} epochs")

if report.safe_epoch_lag_max > 10:
    print(f"  ⚠️ High epoch lag detected - some threads may be stalling")
else:
    print(f"  ✅ Epoch lag is within normal range")

## 6. Reclamation Analysis

In [ ]:
print("### Reclamation Statistics ###")
print(f"\nTotal retired: {report.total_retired:,} nodes")
print(f"Total reclaimed: {report.total_reclaimed:,} nodes")
print(f"Currently pending: {report.pending_count:,} nodes ({report.pending_bytes:,} bytes)")
print(f"Peak pending: {report.peak_pending_count:,} nodes ({report.peak_pending_bytes:,} bytes)")

reclaim_rate = report.total_reclaimed / report.total_retired * 100 if report.total_retired > 0 else 0
print(f"Reclamation efficiency: {reclaim_rate:.1f}%")

print(f"\n### Reclamation Latency (retire → free) ###")
print(f"P50:   {report.reclaim_latency_p50 / 1000:>8.1f} µs")
print(f"P95:   {report.reclaim_latency_p95 / 1000:>8.1f} µs")
print(f"P99:   {report.reclaim_latency_p99 / 1000:>8.1f} µs")
print(f"P99.9: {report.reclaim_latency_p999 / 1000:>8.1f} µs")
print(f"Max:   {report.reclaim_latency_max / 1000:>8.1f} µs")

print(f"\n### Poll Statistics ###")
print(f"Total polls: {report.poll_count:,}")
print(f"Nodes per poll (avg): {report.nodes_per_poll_avg:.1f}")
print(f"Nodes per poll (max): {report.nodes_per_poll_max}")
print(f"Empty polls: {report.empty_poll_pct:.1f}%")

## 7. Memory Pressure Analysis

In [ ]:
print("### Memory Pressure ###")
print(f"\nMemory bound utilization: {report.memory_bound_utilization:.1%}")

# Calculate theoretical bounds
T = len(report.thread_stats)
R = 64  # Default retire threshold
theoretical_max = 3 * T * R

print(f"\nTheoretical bounds:")
print(f"  Threads (T): {T}")
print(f"  Retire threshold (R): {R}")
print(f"  Theoretical max (3×T×R): {theoretical_max:,} nodes")
print(f"  Actual peak: {report.peak_pending_count:,} nodes")
print(f"  Headroom: {((theoretical_max - report.peak_pending_count) / theoretical_max * 100):.1f}%")

if report.memory_bound_utilization > 0.8:
    print(f"\n⚠️ Memory pressure is HIGH")
    print(f"   Consider: increasing retire threshold, reducing thread count, or using DEBRA+")
elif report.memory_bound_utilization > 0.5:
    print(f"\n🟡 Memory pressure is MODERATE")
else:
    print(f"\n✅ Memory pressure is LOW")

## 8. Stall Analysis

In [ ]:
print("### Stall Analysis ###")
print(f"\nTotal stall events: {report.stall_count}")
print(f"Total stall time: {report.total_stall_time_ns / 1e6:.1f} ms")

if report.stall_events:
    print(f"\nStall events:")
    print(f"{'Time':>8} {'Thread':>8} {'At Epoch':>10} {'Global':>10} {'Lag':>6} {'Duration':>10} {'Resolution'}")
    print("-" * 75)
    
    for stall in report.stall_events:
        duration = f"{stall.duration_ns / 1e6:.1f}ms" if stall.duration_ns else "ongoing"
        print(f"{stall.timestamp:>8.2f} {stall.stalled_thread_id:>8} {stall.stalled_at_epoch:>10,} "
              f"{stall.global_epoch:>10,} {stall.epoch_lag:>6} {duration:>10} {stall.resolution}")
    
    # Analyze stall patterns
    stall_by_thread = {}
    for stall in report.stall_events:
        tid = stall.stalled_thread_id
        if tid not in stall_by_thread:
            stall_by_thread[tid] = []
        stall_by_thread[tid].append(stall)
    
    print(f"\nStalls by thread:")
    for tid, stalls in sorted(stall_by_thread.items(), key=lambda x: -len(x[1])):
        total_time = sum(s.duration_ns or 0 for s in stalls)
        print(f"  Thread {tid}: {len(stalls)} stalls, {total_time / 1e6:.1f}ms total")
else:
    print("\n✅ No stall events detected")

## 9. Per-Thread Statistics

In [ ]:
print("### Per-Thread SMR Statistics ###\n")

# Sort by retire count
sorted_threads = sorted(report.thread_stats, key=lambda x: -x.retire_count)

print(f"{'Thread':<12} {'Enter':>8} {'Retire':>8} {'Reclaim':>8} {'Avg CS':>10} {'Stalls':>7}")
print("-" * 60)

for ts in sorted_threads:
    name = ts.thread_name or f"Thread-{ts.thread_id}"
    avg_cs = f"{ts.avg_cs_time_ns / 1000:.1f}µs"
    print(f"{name:<12} {ts.enter_count:>8,} {ts.retire_count:>8,} {ts.reclaim_count:>8,} "
          f"{avg_cs:>10} {ts.stall_count:>7}")

# Find problematic threads
print(f"\n### Thread Health ###")
for ts in sorted_threads:
    name = ts.thread_name or f"Thread-{ts.thread_id}"
    issues = []
    
    if ts.stall_count > 0:
        issues.append(f"{ts.stall_count} stalls")
    if ts.caused_stall_epochs > 5:
        issues.append(f"blocked {ts.caused_stall_epochs} epoch advances")
    if ts.max_cs_time_ns > 1000000:  # > 1ms
        issues.append(f"max CS time {ts.max_cs_time_ns / 1e6:.1f}ms")
    
    if issues:
        print(f"  ⚠️ {name}: {', '.join(issues)}")

if all(ts.stall_count == 0 and ts.caused_stall_epochs < 5 for ts in sorted_threads):
    print(f"  ✅ All threads operating normally")

## 10. Visualizations

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    PLOTTING_AVAILABLE = True
except ImportError:
    PLOTTING_AVAILABLE = False
    print("matplotlib not available")

In [ ]:
if PLOTTING_AVAILABLE:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Reclamation latency distribution
    ax1 = axes[0, 0]
    percentiles = ['P50', 'P95', 'P99', 'P99.9', 'Max']
    latencies = [
        report.reclaim_latency_p50 / 1000,
        report.reclaim_latency_p95 / 1000,
        report.reclaim_latency_p99 / 1000,
        report.reclaim_latency_p999 / 1000,
        report.reclaim_latency_max / 1000,
    ]
    colors = ['green', 'yellowgreen', 'orange', 'orangered', 'red']
    ax1.bar(percentiles, latencies, color=colors)
    ax1.set_ylabel('Latency (µs)')
    ax1.set_title('Reclamation Latency Distribution')
    ax1.set_yscale('log')
    
    # 2. Per-thread retire/reclaim
    ax2 = axes[0, 1]
    thread_names = [ts.thread_name or f"T{ts.thread_id}" for ts in report.thread_stats]
    retire_counts = [ts.retire_count for ts in report.thread_stats]
    reclaim_counts = [ts.reclaim_count for ts in report.thread_stats]
    x = np.arange(len(thread_names))
    width = 0.35
    ax2.bar(x - width/2, retire_counts, width, label='Retired', color='coral')
    ax2.bar(x + width/2, reclaim_counts, width, label='Reclaimed', color='mediumseagreen')
    ax2.set_xticks(x)
    ax2.set_xticklabels(thread_names, rotation=45, ha='right')
    ax2.set_ylabel('Count')
    ax2.set_title('Retire vs Reclaim by Thread')
    ax2.legend()
    
    # 3. Critical section time
    ax3 = axes[1, 0]
    avg_cs_times = [ts.avg_cs_time_ns / 1000 for ts in report.thread_stats]
    max_cs_times = [ts.max_cs_time_ns / 1000 for ts in report.thread_stats]
    ax3.bar(x - width/2, avg_cs_times, width, label='Avg', color='steelblue')
    ax3.bar(x + width/2, max_cs_times, width, label='Max', color='indianred')
    ax3.set_xticks(x)
    ax3.set_xticklabels(thread_names, rotation=45, ha='right')
    ax3.set_ylabel('Time (µs)')
    ax3.set_title('Critical Section Duration by Thread')
    ax3.legend()
    ax3.set_yscale('log')
    
    # 4. Memory pressure gauge
    ax4 = axes[1, 1]
    utilization = report.memory_bound_utilization
    colors_gauge = ['green' if utilization < 0.5 else 'orange' if utilization < 0.8 else 'red']
    ax4.barh(['Memory Bound\nUtilization'], [utilization], color=colors_gauge, height=0.5)
    ax4.barh(['Memory Bound\nUtilization'], [1 - utilization], left=[utilization], 
             color='lightgray', height=0.5)
    ax4.set_xlim(0, 1)
    ax4.set_xlabel('Utilization')
    ax4.set_title(f'Memory Pressure: {utilization:.1%}')
    ax4.axvline(x=0.5, color='orange', linestyle='--', alpha=0.5)
    ax4.axvline(x=0.8, color='red', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig('smr_profile_charts.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Charts saved to smr_profile_charts.png")

## 11. Recommendations

In [ ]:
def generate_smr_recommendations(report):
    recommendations = []
    
    # Check epoch lag
    if report.safe_epoch_lag_max > 20:
        recommendations.append({
            'severity': 'error',
            'category': 'Epoch Lag',
            'issue': f'Very high epoch lag: {report.safe_epoch_lag_max} epochs',
            'recommendation': 'Consider using DEBRA+ for automatic stall handling',
        })
    elif report.safe_epoch_lag_max > 10:
        recommendations.append({
            'severity': 'warning',
            'category': 'Epoch Lag',
            'issue': f'High epoch lag: {report.safe_epoch_lag_max} epochs',
            'recommendation': 'Identify stalling threads and reduce critical section time',
        })
    
    # Check reclamation latency
    if report.reclaim_latency_p99 > 1000000:  # > 1ms
        recommendations.append({
            'severity': 'warning',
            'category': 'Latency',
            'issue': f'High reclamation latency P99: {report.reclaim_latency_p99/1000:.0f}µs',
            'recommendation': 'Increase poll frequency or reduce retire threshold',
        })
    
    # Check memory pressure
    if report.memory_bound_utilization > 0.8:
        recommendations.append({
            'severity': 'error',
            'category': 'Memory',
            'issue': f'Memory bound utilization: {report.memory_bound_utilization:.1%}',
            'recommendation': 'Risk of hitting memory bound; reduce thread count or use DEBRA+',
        })
    elif report.memory_bound_utilization > 0.5:
        recommendations.append({
            'severity': 'warning',
            'category': 'Memory',
            'issue': f'Moderate memory pressure: {report.memory_bound_utilization:.1%}',
            'recommendation': 'Monitor for increases under higher load',
        })
    
    # Check stalls
    if report.stall_count > 10:
        recommendations.append({
            'severity': 'warning',
            'category': 'Stalls',
            'issue': f'{report.stall_count} stall events detected',
            'recommendation': 'Reduce blocking operations in critical sections',
        })
    
    # Check empty polls
    if report.empty_poll_pct > 30:
        recommendations.append({
            'severity': 'info',
            'category': 'Efficiency',
            'issue': f'{report.empty_poll_pct:.0f}% of polls found nothing to reclaim',
            'recommendation': 'Consider increasing retire threshold to reduce overhead',
        })
    
    # Check for good health
    if not recommendations:
        recommendations.append({
            'severity': 'success',
            'category': 'Overall',
            'issue': 'SMR operating normally',
            'recommendation': 'No optimization needed',
        })
    
    return recommendations

recommendations = generate_smr_recommendations(report)

severity_icons = {
    'error': '\U0001F534',
    'warning': '\U0001F7E0',
    'info': '\U0001F7E1',
    'success': '\u2705',
}

print("### SMR Recommendations ###\n")
for rec in recommendations:
    icon = severity_icons.get(rec['severity'], '')
    print(f"{icon} [{rec['category']}] {rec['issue']}")
    print(f"   → {rec['recommendation']}\n")

## Summary

This notebook analyzed SMR behavior including:

1. **Epoch Analysis** - Advancement rate and lag
2. **Reclamation Metrics** - Latency, efficiency, poll statistics
3. **Memory Pressure** - Utilization vs theoretical bounds
4. **Stall Analysis** - Identified blocking threads
5. **Per-Thread Stats** - Critical section times, retire/reclaim counts
6. **Visualizations** - Charts for quick understanding
7. **Recommendations** - Actionable optimization suggestions

For comparing SMR schemes, see `memory_subsystem_comparison.ipynb`.